In [1]:
#백테스팅 결과 참고 후, 수정 필요
#유니버스 - 소형주
#세 가지 팩터만 활용하기로 함

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pykrx import stock

def perform_screening(end_date):
    end_date_datetime = datetime.strptime(end_date, "%Y%m%d")
    start_date_datetime = end_date_datetime - timedelta(days=30)
    start_date = start_date_datetime.strftime("%Y%m%d")

    # 시가총액 데이터 가져오기
    
    kospi_small = stock.get_index_portfolio_deposit_file("1004")

    market_cap_data = stock.get_market_cap(end_date, market="ALL")
    market_cap_data = market_cap_data[['종가', '시가총액']]

    filtered_market_cap_data = market_cap_data.loc[kospi_small]
    
    gains = []
    filtered_tickers = filtered_market_cap_data.index

    for ticker in filtered_tickers:
        df = stock.get_market_ohlcv(start_date, end_date, ticker, "m")
        gain = ((df.iloc[1,3] - df.iloc[0,3]) / df.iloc[0,3]) * 100
        gains.append(gain)

    result_df = pd.DataFrame({'ticker': filtered_tickers, 'gain': gains})
    result_df.set_index('ticker', inplace=True)
    gain_25 = result_df['gain'].quantile(0.25)
    selected_rows = result_df['gain'] < gain_25
    temp_df = result_df[selected_rows]
    selected_tickers = temp_df.index.tolist()

    # 평균값 계산
    revisions = []

    for ticker in selected_tickers:
        data = stock.get_market_ohlcv(start_date, end_date, ticker, "d")
        mean_price = data['종가'].mean()
        close_temp = stock.get_market_ohlcv(start_date, end_date, ticker, "d")
        close = close_temp.iloc[0,3]
        revision = (close / mean_price) * 100
        revisions.append(revision)

    result_df = pd.DataFrame({'ticker': selected_tickers, 'revision': revisions})
    result_df.set_index('ticker', inplace=True)
    revision_25 = result_df['revision'].quantile(0.25)
    selected_rows = result_df['revision'] < revision_25
    revision_df = result_df[selected_rows]

    # 평균값 계산
    mean_values = []

    for ticker in revision_df.index:
        data1 = stock.get_market_trading_value_by_date(start_date, end_date, ticker)
        data2 = stock.get_market_cap(start_date, end_date, ticker)['시가총액']
        data1['개인순매수'] = (data1['개인'] / data2) * 100
        mean = data1['개인순매수'].mean()
        mean_values.append(mean)

    final_df = pd.DataFrame({'ticker': revision_df.index, 'mean': mean_values})
    final_df.set_index('ticker', inplace=True)
    final_75 = final_df['mean'].quantile(0.75)
    selected_rows = final_df['mean'] > final_75
    screening = final_df[selected_rows]

    return screening

# 스크리닝 함수 호출
end_date = "20240322"
screened_stocks = perform_screening(end_date)
print(screened_stocks.index)

Index(['007810', '010690', '001250', '163560', '011300', '004770', '011420',
       '033250'],
      dtype='object', name='ticker')


### 백테스트 코드

In [3]:
# 인덱스들 종가 불러오기
final_values = []
for ticker in screened_stocks.index:
    data = stock.get_market_ohlcv("20240322", "20240322", ticker, "d")
    final = data.iloc[0,3]
    final_values.append(final)

final_df = pd.DataFrame({'ticker': screened_stocks.index, '종가': final_values})
cash = 100000000 # 모의투자 시작금액인 1억으로 시작하겠습니다.
money = cash

if len(screened_stocks.index)==0:
    allocation=0
else:
    allocation = money / len(screened_stocks.index) # 동일 비중이기 때문에 보유 현금 / 투자할 종목수 로 나눠줍니다.

final_df['매수 수량'] = allocation / final_df['종가']
final_df

,ticker,종가,매수 수량
0,007810,18100,690.607735
1,010690,11460,1090.750436
2,001250,2760,4528.985507
3,163560,8300,1506.024096
4,011300,1238,10096.930533
5,004770,2150,5813.953488
6,011420,2505,4990.019960
7,033250,1365,9157.509158


In [112]:
import FinanceDataReader as fdr #벤치마킹 지수

ref = fdr.DataReader(
    symbol = 'KS11',
    start = '20190319',
    end = '20240322')

print(ref)

               Open     High      Low    Close     Volume  Change  UpDown  \
Date                                                                        
2019-03-19  2183.03  2183.76  2171.05  2177.62  319702553 -0.0009       2   
2019-03-20  2171.41  2177.19  2152.21  2177.10  317606686 -0.0002       2   
2019-03-21  2183.11  2200.32  2173.70  2184.88  342747152  0.0036       1   
2019-03-22  2192.72  2195.42  2178.77  2186.95  333617567  0.0009       1   
2019-03-25  2158.80  2159.25  2143.70  2144.86  307071247 -0.0192       2   
...             ...      ...      ...      ...        ...     ...     ...   
2024-03-18  2678.52  2688.07  2665.28  2685.84  408182483  0.0071       1   
2024-03-19  2673.57  2673.57  2645.14  2656.17  695865617 -0.0110       2   
2024-03-20  2680.70  2692.52  2674.09  2690.14  720451805  0.0128       1   
2024-03-21  2731.18  2755.45  2724.25  2754.86  825226939  0.0241       1   
2024-03-22  2751.15  2763.06  2738.18  2748.56  509545275 -0.0023       2   

In [114]:
import pandas as pd
import numpy as np

# 종가를 기준으로 수익률, 누적 수익률, 로그 수익률, 누적 로그 수익률을 계산하는 함수 정의
def calculate_returns(df):
    # 수익률 계산
    returns = df['Close'].pct_change() * 100  # 변화율을 백분율로 표현
    cum_returns = (1 + returns / 100).cumprod() - 1  # 누적 수익률 계산
    
    # 로그 수익률 계산
    log_returns = np.log(1 + returns / 100)
    cum_log_returns = log_returns.cumsum()  # 누적 로그 수익률 계산
    
    # 데이터프레임으로 결과 반환
    result_df = pd.DataFrame({
        '수익률': returns,
        '누적 수익률': cum_returns,
        '로그 수익률': log_returns,
        '누적 로그 수익률': cum_log_returns
    }, index=df.index)
    
    return result_df

# 주어진 데이터프레임에 대해 수익률 계산
ref_returns = calculate_returns(ref)

ref_returns

,수익률,누적 수익률,로그 수익률,누적 로그 수익률
Date,,,,
2019-03-19,NaN,NaN,NaN,NaN
2019-03-20,-0.023879,-0.000239,-0.000239,-0.000239
2019-03-21,0.357356,0.003334,0.003567,0.003328
2019-03-22,0.094742,0.004284,0.000947,0.004275
2019-03-25,-1.924598,-0.015044,-0.019434,-0.015158
...,...,...,...,...
2024-03-18,0.712454,0.233383,0.007099,0.209761
2024-03-19,-1.104682,0.219758,-0.011108,0.198653
2024-03-20,1.278909,0.235358,0.012708,0.211361


In [27]:
data = stock.get_market_ohlcv("20190319", "20240322", "007810", "m")
print(data)

               시가     고가     저가     종가       거래량
날짜                                              
2019-03-31   6230   6300   5700   5760    243255
2019-04-30   5690   6770   5690   6050   1446544
2019-05-31   6050   6350   5100   5250    845269
2019-06-30   5200   5680   5120   5500    416010
2019-07-31   5510   5550   4780   4950    484010
...           ...    ...    ...    ...       ...
2023-11-30  13620  17070  13620  16670   3333551
2023-12-31  16670  22550  15610  19990  51265735
2024-01-31  19940  21750  17850  17930  20446843
2024-02-29  17620  22250  16400  18900  18214250
2024-03-31  19410  21000  17830  18100   5879171

[61 rows x 5 columns]


In [120]:
stock_codes = screened_stocks.index

result_data = []

for code in stock_codes:
    data = stock.get_market_ohlcv("20190319", "20240322", code, "m")
    
    # 데이터가 존재하는 경우에만 계산하고 결과 리스트에 추가합니다.
    if not data.empty:
        # 종가와 시가의 차이를 계산하여 새로운 값을 구합니다.
        new_value = data['종가']
        
        # 결과 리스트에 종목 코드를 인덱스로 하고, 날짜를 칼럼으로 하는 데이터프레임을 추가합니다.
        result_data.append(pd.DataFrame(new_value.values, index=data.index, columns=[code]))

final_result = pd.concat(result_data, axis=1)
print(final_result)

            007810  010690  001250  163560  011300  004770  011420  033250
날짜                                                                        
2019-03-31    5760    2420    2555    6357     643    2795    1500     797
2019-04-30    6050    3530    2660    6530     683    3180    1500    1649
2019-05-31    5250    2800    2615    6319     628    3235    1750    2074
2019-06-30    5500    2790    2810    6184     640    3265    2130    2194
2019-07-31    4950    2435    2445    5712     598    3100    2970    1575
...            ...     ...     ...     ...     ...     ...     ...     ...
2023-11-30   16670   11750    2575    6210    1456    2425    2685    2770
2023-12-31   19990   13680    2550    6200    1631    2365    2635    1938
2024-01-31   17930   11600    2575    6220    1341    2260    2405    1515
2024-02-29   18900   12510    2925    8990    1391    2235    3185    1650
2024-03-31   18100   11460    2760    8300    1238    2150    2505    1365

[61 rows x 8 columns]


In [99]:
import pandas as pd
import numpy as np

stock_codes = screened_stocks.index

result_data = []

for code in stock_codes:
    data = stock.get_market_ohlcv("20190319", "20240322", code, "m")
    
    # 데이터가 존재하는 경우에만 계산하고 결과 리스트에 추가합니다.
    if not data.empty:
        # 종가와 시가의 차이를 계산하여 새로운 값을 구합니다.
        new_value = data['종가']
        
        # 수익률 계산
        returns = new_value.pct_change() * 100  # 변화율을 백분율로 표현
        cum_returns = (1 + returns / 100).cumprod() - 1  # 누적 수익률 계산
        
        # 로그 수익률 계산
        log_returns = np.log(1 + returns / 100)
        cum_log_returns = log_returns.cumsum()  # 누적 로그 수익률 계산
        
        # 결과 리스트에 종목 코드를 인덱스로 하고, 날짜를 칼럼으로 하는 데이터프레임을 추가합니다.
        result_df = pd.DataFrame(new_value.values, index=data.index, columns=[code])
        result_df['수익률'] = returns  # 수익률 칼럼 추가
        result_df['누적 수익률'] = cum_returns  # 누적 수익률 칼럼 추가
        result_df['로그 수익률'] = log_returns  # 로그 수익률 칼럼 추가
        result_df['누적 로그 수익률'] = cum_log_returns  # 누적 로그 수익률 칼럼 추가
        result_data.append(result_df)
        
final_result = pd.concat(result_data, axis=1)
final_result

,007810,수익률,누적 수익률,로그 수익률,누적 로그 수익률,010690,수익률,누적 수익률,로그 수익률,누적 로그 수익률,...,011420,수익률,누적 수익률,로그 수익률,누적 로그 수익률,033250,수익률,누적 수익률,로그 수익률,누적 로그 수익률
날짜,,,,,,,,,,,,,,,,,,,,,
2019-03-31,5760,NaN,NaN,NaN,NaN,2420,NaN,NaN,NaN,NaN,...,1500,NaN,NaN,NaN,NaN,797,NaN,NaN,NaN,NaN
2019-04-30,6050,5.034722,0.050347,0.049121,0.049121,3530,45.867769,0.458678,0.377530,0.377530,...,1500,0.000000,0.000000,0.000000,0.000000,1649,106.900878,1.069009,0.727070,0.727070
2019-05-31,5250,-13.223140,-0.088542,-0.141830,-0.092709,2800,-20.679887,0.157025,-0.231678,0.145852,...,1750,16.666667,0.166667,0.154151,0.154151,2074,25.773196,1.602258,0.229310,0.956380
2019-06-30,5500,4.761905,-0.045139,0.046520,-0.046189,2790,-0.357143,0.152893,-0.003578,0.142274,...,2130,21.714286,0.420000,0.196506,0.350657,2194,5.785921,1.752823,0.056247,1.012627
2019-07-31,4950,-10.000000,-0.140625,-0.105361,-0.151550,2435,-12.724014,0.006198,-0.136095,0.006179,...,2970,39.436620,0.980000,0.332440,0.683097,1575,-28.213309,0.976161,-0.331471,0.681156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-30,16670,23.025830,1.894097,0.207224,1.062673,11750,16.683217,3.855372,0.154293,1.580086,...,2685,61.067786,0.790000,0.476655,0.582216,2770,89.726027,2.475533,0.640411,1.245748
2023-12-31,19990,19.916017,2.470486,0.181621,1.244295,13680,16.425532,4.652893,0.152082,1.732167,...,2635,-1.862197,0.756667,-0.018798,0.563418,1938,-30.036101,1.431619,-0.357191,0.888557
2024-01-31,17930,-10.305153,2.112847,-0.108757,1.135538,11600,-15.204678,3.793388,-0.164930,1.567238,...,2405,-8.728653,0.603333,-0.091333,0.472085,1515,-21.826625,0.900878,-0.246241,0.642316


In [105]:
# 각 칼럼들의 평균을 계산하여 새로운 데이터프레임으로 만듭니다.
mean_result = final_result.groupby(level=0, axis=1).mean()
mean_result = mean_result[['누적 로그 수익률', '누적 수익률', '로그 수익률', '수익률']]
print(mean_result)

            누적 로그 수익률    누적 수익률    로그 수익률        수익률
날짜                                                  
2019-03-31        NaN       NaN       NaN        NaN
2019-04-30   0.176281  0.230787  0.176281  23.078726
2019-05-31   0.162935  0.248626 -0.013345  -0.338660
2019-06-30   0.209708  0.314582  0.046772   5.007965
2019-07-31   0.137363  0.214545 -0.072345  -5.467341
...               ...       ...       ...        ...
2023-11-30   0.641303  1.266464  0.174444  23.457333
2023-12-31   0.645653  1.333464  0.004349   1.607044
2024-01-31   0.540718  1.036356 -0.104934  -9.622798
2024-02-29   0.667686  1.270776  0.126968  14.418305
2024-03-31   0.560946  1.042629 -0.106740  -9.920941

[61 rows x 4 columns]


In [117]:
import plotly.graph_objs as go

# mean_result 데이터프레임으로부터 날짜와 각 열 데이터 추출
dates = mean_result.index
cumulative_log_returns = mean_result['누적 로그 수익률']
cumulative_returns = mean_result['누적 수익률']
cumulative_log_returns1 = ref_returns['누적 로그 수익률']
cumulative_returns1 = ref_returns['누적 수익률']
# 라인 그래프 생성
trace1 = go.Scatter(x=dates, y=cumulative_log_returns, mode='lines', name='포트폴리오 누적 로그 수익률')
trace2 = go.Scatter(x=dates, y=cumulative_returns, mode='lines', name='포트폴리오 누적 수익률')
trace3 = go.Scatter(x=dates, y=cumulative_log_returns1, mode='lines', name='코스피 누적 로그 수익률')
trace4 = go.Scatter(x=dates, y=cumulative_returns1, mode='lines', name='코스피 누적 수익률')

# 그래프 레이아웃 설정
layout = go.Layout(title='<b> 누적 및 로그 수익률 <b>', xaxis=dict(title='날짜'), yaxis=dict(title='수익률'))

# 그래프 생성 및 표시
fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
fig.show()

In [119]:
import plotly.graph_objs as go
import plotly.io as pio

# Set Seaborn template
pio.templates["seaborn"] = go.layout.Template(
    layout=go.Layout(
        font=dict(family="Arial, sans-serif"),
        title=dict(font=dict(size=20)),
        xaxis=dict(title_font=dict(size=16)),
        yaxis=dict(title_font=dict(size=16)),
        legend=dict(font=dict(size=14)),
    )
)
pio.templates.default = "seaborn"

# mean_result 데이터프레임으로부터 날짜와 각 열 데이터 추출
dates = mean_result.index
cumulative_log_returns = mean_result['누적 로그 수익률']
cumulative_returns = mean_result['누적 수익률']
cumulative_log_returns1 = ref_returns['누적 로그 수익률']
cumulative_returns1 = ref_returns['누적 수익률']

# 라인 그래프 생성
trace1 = go.Scatter(x=dates, y=cumulative_log_returns, mode='lines', name='포트폴리오 누적 로그 수익률')
trace2 = go.Scatter(x=dates, y=cumulative_returns, mode='lines', name='포트폴리오 누적 수익률')
trace3 = go.Scatter(x=dates, y=cumulative_log_returns1, mode='lines', name='코스피 누적 로그 수익률')
trace4 = go.Scatter(x=dates, y=cumulative_returns1, mode='lines', name='코스피 누적 수익률')

# 그래프 레이아웃 설정
layout = go.Layout(
    title='<b>누적 및 로그 수익률</b>',
    xaxis=dict(title='날짜', showgrid=True, tickfont=dict(size=12)),
    yaxis=dict(title='수익률', showgrid=True, tickfont=dict(size=12)),
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255, 255, 255, 0.5)', bordercolor='rgba(0, 0, 0, 0.5)'),
    plot_bgcolor='rgba(0, 0, 0, 0)'
)

# 그래프 생성 및 표시
fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
fig.show()

In [128]:
#백테스트 전체 코드 
import FinanceDataReader as fdr #벤치마킹 지수

ref = fdr.DataReader(
    symbol = 'KS11',
    start = '20180319',
    end = '20240322')

import pandas as pd
import numpy as np

# 종가를 기준으로 수익률, 누적 수익률, 로그 수익률, 누적 로그 수익률을 계산하는 함수 정의
def calculate_returns(df):
    # 수익률 계산
    returns = df['Close'].pct_change() * 100  # 변화율을 백분율로 표현
    cum_returns = (1 + returns / 100).cumprod() - 1  # 누적 수익률 계산
    
    # 로그 수익률 계산
    log_returns = np.log(1 + returns / 100)
    cum_log_returns = log_returns.cumsum()  # 누적 로그 수익률 계산
    
    # 데이터프레임으로 결과 반환
    result_df = pd.DataFrame({
        '수익률': returns,
        '누적 수익률': cum_returns,
        '로그 수익률': log_returns,
        '누적 로그 수익률': cum_log_returns
    }, index=df.index)
    
    return result_df

# 주어진 데이터프레임에 대해 수익률 계산
ref_returns = calculate_returns(ref)

#포트폴리오 계산
stock_codes = screened_stocks.index

result_data = []

for code in stock_codes:
    data = stock.get_market_ohlcv("20180319", "20240322", code, "m")
    
    # 데이터가 존재하는 경우에만 계산하고 결과 리스트에 추가합니다.
    if not data.empty:
        # 종가와 시가의 차이를 계산하여 새로운 값을 구합니다.
        new_value = data['종가']
        
        # 결과 리스트에 종목 코드를 인덱스로 하고, 날짜를 칼럼으로 하는 데이터프레임을 추가합니다.
        result_data.append(pd.DataFrame(new_value.values, index=data.index, columns=[code]))

final_result = pd.concat(result_data, axis=1)

stock_codes = screened_stocks.index

result_data = []

for code in stock_codes:
    data = stock.get_market_ohlcv("20180319", "20240322", code, "m")
    
    # 데이터가 존재하는 경우에만 계산하고 결과 리스트에 추가합니다.
    if not data.empty:
        # 종가와 시가의 차이를 계산하여 새로운 값을 구합니다.
        new_value = data['종가']
        
        # 수익률 계산
        returns = new_value.pct_change() * 100  # 변화율을 백분율로 표현
        cum_returns = (1 + returns / 100).cumprod() - 1  # 누적 수익률 계산
        
        # 로그 수익률 계산
        log_returns = np.log(1 + returns / 100)
        cum_log_returns = log_returns.cumsum()  # 누적 로그 수익률 계산
        
        # 결과 리스트에 종목 코드를 인덱스로 하고, 날짜를 칼럼으로 하는 데이터프레임을 추가합니다.
        result_df = pd.DataFrame(new_value.values, index=data.index, columns=[code])
        result_df['수익률'] = returns  # 수익률 칼럼 추가
        result_df['누적 수익률'] = cum_returns  # 누적 수익률 칼럼 추가
        result_df['로그 수익률'] = log_returns  # 로그 수익률 칼럼 추가
        result_df['누적 로그 수익률'] = cum_log_returns  # 누적 로그 수익률 칼럼 추가
        result_data.append(result_df)
        
final_result = pd.concat(result_data, axis=1)
final_result

mean_result = final_result.groupby(level=0, axis=1).mean()
mean_result = mean_result[['누적 로그 수익률', '누적 수익률', '로그 수익률', '수익률']]

In [130]:
import plotly.graph_objs as go
import plotly.io as pio

# Set Seaborn template
pio.templates["seaborn"] = go.layout.Template(
    layout=go.Layout(
        font=dict(family="Arial, sans-serif"),
        title=dict(font=dict(size=20)),
        xaxis=dict(title_font=dict(size=16)),
        yaxis=dict(title_font=dict(size=16)),
        legend=dict(font=dict(size=12)),
    )
)
pio.templates.default = "seaborn"

# mean_result 데이터프레임으로부터 날짜와 각 열 데이터 추출
dates = mean_result.index
cumulative_log_returns = mean_result['누적 로그 수익률']
cumulative_returns = mean_result['누적 수익률']
cumulative_log_returns1 = ref_returns['누적 로그 수익률']
cumulative_returns1 = ref_returns['누적 수익률']

# 라인 그래프 생성
trace1 = go.Scatter(x=dates, y=cumulative_log_returns, mode='lines', name='포트폴리오 누적 로그 수익률')
trace2 = go.Scatter(x=dates, y=cumulative_returns, mode='lines', name='포트폴리오 누적 수익률')
trace3 = go.Scatter(x=dates, y=cumulative_log_returns1, mode='lines', name='코스피 누적 로그 수익률')
trace4 = go.Scatter(x=dates, y=cumulative_returns1, mode='lines', name='코스피 누적 수익률')

# 그래프 레이아웃 설정
layout = go.Layout(
    title='<b>누적 및 로그 수익률</b>',
    xaxis=dict(title='날짜', tickmode='auto', dtick='M12', tickformat='%Y', showgrid=True, tickfont=dict(size=12)),
    yaxis=dict(title='수익률', showgrid=True, tickfont=dict(size=12)),
    legend=dict(x=0.5, y=1.1, bgcolor='rgba(255, 255, 255, 0.5)', bordercolor='rgba(0, 0, 0, 0.5)'),
    plot_bgcolor='rgba(0, 0, 0, 0)'
)

# 그래프 생성 및 표시
fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
fig.show()